# Spectral Analysis Reproduction

This notebook reproduces the key experimental results from the paper. It loads the pre-computed spectral sweep data and baseline perplexity/logprob data to generate the comparative metrics (AUC, Precision, Recall) reported in the tables.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import roc_auc_score, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')

# Set paths
DATA_DIR = Path('../data')
BASELINE_DIR = DATA_DIR / 'baselines'
SWEEP_DIR = DATA_DIR / 'categories_sweeps'

## 1. Helper Functions

In [ ]:
def get_metrics(y_true, y_score):
    """Calculate AUC, Best Precision, and Best Recall."""
    try:
        auc = roc_auc_score(y_true, y_score)
    except ValueError:
        auc = 0.5

    precisions, recalls, thresholds = precision_recall_curve(y_true, y_score)
    f1s = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
    best_idx = np.argmax(f1s)
    
    return auc, precisions[best_idx], recalls[best_idx]

def analyze_discriminator(df, score_col, label_col='is_hallucination', invert=False):
    """Analyze a single feature as a discriminator."""
    y = df[label_col].values
    scores = df[score_col].values
    
    if invert:
        scores = -scores
        
    # Auto-detect direction if not strictly specified, but for reproduction we usually fix it.
    # Here, we check correlation to align with the paper's 'best direction' approach.
    corr = np.corrcoef(y, scores)[0,1]
    if np.isnan(corr): corr = 0
    
    final_scores = scores if corr > 0 else -scores
    direction = '+' if corr > 0 else '-'
    
    auc, prec, rec = get_metrics(y, final_scores)
    return auc, prec, rec, direction

## 2. Baseline Analysis (PPL & LogProb)

Reproduces the PPL and Min LogProb baselines table.

In [ ]:
baseline_files = list(BASELINE_DIR.glob('*_ppl.csv'))
baseline_results = []

for f in baseline_files:
    df = pd.read_csv(f)
    name = f.stem.replace('_ppl', '').replace('_', ' ').title()
    
    # PPL Analysis
    p_auc, p_prec, p_rec, p_dir = analyze_discriminator(df, 'ppl')
    
    # Min LogProb Analysis
    # Note: paper uses 'logprob_min' for stronger baselines
    l_auc, l_prec, l_rec, l_dir = analyze_discriminator(df, 'logprob_min')
    
    baseline_results.append({
        'Dataset': name,
        'N': len(df),
        'Hallucinations': df['is_hallucination'].sum(),
        'PPL AUC': p_auc,
        'PPL Prec': p_prec,
        'PPL Rec': p_rec,
        'LogProb AUC': l_auc,
        'LogProb Prec': l_prec,
        'LogProb Rec': l_rec
    })

df_base = pd.DataFrame(baseline_results)
display(df_base.round(4))

## 3. Spectral Sweep Analysis

Reproduces the per-category spectral analysis (HFER, Fiedler, etc.).

In [ ]:
sweep_files = list(SWEEP_DIR.glob('*.csv'))
sweep_results = []

spectral_metrics = ['HFER', 'Fiedler_E_val', 'smoothness_entropy', 'spectral_entropy']

for f in sweep_files:
    try:
        df = pd.read_csv(f)
        if 'is_hallucination' not in df.columns:
            continue
            
        name = f.stem.replace('mistral_', '').capitalize()
        
        row = {'Category': name, 'N': len(df)}
        
        for metric in spectral_metrics:
            auc, prec, rec, _ = analyze_discriminator(df, metric)
            row[f'{metric} AUC'] = auc
            
        sweep_results.append(row)
    except Exception as e:
        print(f"Skipping {f.name}: {e}")

df_sweep = pd.DataFrame(sweep_results)
display(df_sweep.round(4))